In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
from preprocess import DB
import matplotlib.pyplot as plt

In [2]:
db = DB(db_route = '../data/database.sqlite')

In [5]:
#df = db.combineCardFoulDF()

In [19]:
base_elapsed = pd.DataFrame({'elapsed_foul' : list(range(120))})

In [62]:
df1 = pd.merge(base_elapsed, db.unravelFoulDF(2), how = 'left', on = 'elapsed_foul').iloc[:20]

In [66]:
df1[~df1['match_id'].isna()]['match_id'].iloc[0]

1024845.0

In [67]:
for key in list(df1.dtypes.keys()):
    if df1.dtypes[key] == np.dtype('float64'):
        if key == 'match_id':
            mid = df1[~df1['match_id'].isna()]['match_id'].iloc[0]
            df1 = df1.fillna({key : mid}).astype({str(key) : int})
        df1 = df1.fillna({key : 0}).astype({str(key) : int})

In [68]:
df1

,elapsed_foul,match_id,foul_id,foulscommitted,event_incident_typefk_foul,foul_player1,foul_player2,foul_player1_id,foul_player2_id,foul_team,foul_type,foul_subtype
0,0,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
1,1,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
2,2,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
3,3,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
4,4,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
5,5,1024845,1791936,1,37,Steven Taylor,Javier Hernandez,24228,70409,10261,foulcommit,None
6,6,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
7,7,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
8,8,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN
9,9,1024845,0,0,0,NaN,NaN,0,0,0,NaN,NaN


In [69]:
join = pd.merge_asof(
    df1, db.unravelCardDF(2),
    left_on = 'elapsed_foul',
    right_on = 'card_elapsed',
    left_by = 'foul_player1_id',
    right_by = 'card_player_id',
    direction = 'nearest',
    tolerance = 2
)

In [70]:
join#.filter(regex = 'player|elapsed')

,elapsed_foul,match_id_x,foul_id,foulscommitted,event_incident_typefk_foul,foul_player1,foul_player2,foul_player1_id,foul_player2_id,foul_team,...,ycards,event_incident_typefk_card,card_elapsed,card_type,card_subtype,card_team,card_type,card_id,card_player_id,card_player
0,0,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1024845,1791936,1,37,Steven Taylor,Javier Hernandez,24228,70409,10261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,1024845,0,0,0,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
db.unravelFoulDF(2).shape, db.unravelCardDF(2).shape

((26, 12), (6, 13))

In [13]:
db.unravelCardDF(2)

,match_id,rcards,comment,ycards,event_incident_typefk_card,card_elapsed,card_type,card_subtype,card_team,card_type,card_id,card_player_id,card_player
0,1024845,0,y,1,505,27,y,Unsportsmanlike Cond,10261,card,1792331,26163,Hatem Ben Arfa
1,1024845,0,y,1,505,52,y,Unsportsmanlike Cond,10261,card,1792917,26160,Yohan Cabaye
2,1024845,0,y,1,70,54,y,None,10260,card,1792958,150176,Fabio
3,1024845,0,y,1,70,65,y,None,10261,card,1793211,33852,Jonas Gutierrez
5,1024845,0,y,1,70,67,y,None,10261,card,1793265,40565,Danny Guthrie
4,1024845,1,y2,0,26,79,y2,None,10261,card,1793509,33852,Jonas Gutierrez
